Project 43


In [83]:
# Importing Libraries
# import webbrowser
# import time
import tweepy
from tweepy import Stream
from tweepy.auth import OAuthHandler
import os
import pandas as pd


# load keys into keys
#[apiKey,apiKeySecret,bearerTokem,accessToken,accessTokenSecret]
# Consumer Keys: apiKey, apiKeySecret
# Access Keys: accessToken, accessTokenSecret
path =  r'C:\Users\Gumo\Desktop\Git\selfBooks\tweepy.txt'
keys = []
with open(path, mode='r') as w:
    for line in w:
        keys.append(line.split(': ')[1].strip())
access_token = keys[3]
access_secret = keys[4]
consumer_key = keys[0]
consumer_secret = keys[1]


In [86]:
# professor way
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

# access twitter api by keys
auth.set_access_token(access_token,access_secret)
twitterApi = tweepy.API(auth)

# redirect_url = auth.get_authorization_url()
# print(redirect_url)
# webbrowser.open(redirect_url)
try:
    twitterApi.verify_credentials()
    print("twitterApi works")
except:
    print("Please fix me")


# client instance - to post tweet and other access
client = tweepy.Client(consumer_key=consumer_key,consumer_secret=consumer_secret,access_token=access_token,access_token_secret=access_secret)

twitterApi works


In [107]:
# set queries
query1 = "#elonmusk"
query2 = query1 + "-filter:retweets"

In [114]:
# store search in tweets
queryResponse1 = tweepy.Cursor(twitterApi.search_tweets,q=query1,lang="en").items(5)


In [108]:
# store search in tweets
queryResponse2 = tweepy.Cursor(twitterApi.search_tweets,q=query2,lang="en").items(5)

In [116]:
for tweet in queryResponse1:
    print(tweet.user)
    print(tweet.text)
    print('\n')

TypeError: 'User' object is not subscriptable